# GOLD STOCK FORECASTING

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv

C:\Users\acer\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('mae')<0.15):
      print("\nReached 15% mae so cancelling training!")
      self.model.stop_training = True

In [3]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [4]:
def solution_B5():
    # Ganti dengan path lokal file CSV Anda
    file_path = 'goldstock_closing.csv'

    time_step = []
    temps = []

    with open(file_path) as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        next(reader)  # Skip header
        step = 0
        for row in reader:
            temps.append(float(row[2]))  # YOUR CODE HERE)
            time_step.append(step)  # YOUR CODE HERE)
            step=step + 1

    series=np.array(temps) # YOUR CODE HERE

    # Normalization Function. DO NOT CHANGE THIS CODE
    min=np.min(series)
    max=np.max(series)
    series -= min
    series /= max
    time=np.array(time_step)

    # Split data
    split_time = 1700
    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]

    # Dataset parameters
    window_size = 64
    batch_size = 256
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
    print(train_set)
    print(x_train.shape)

    # Membangun model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                               strides=1,
                               activation="relu",
                               padding='causal',
                               input_shape=[None, 1]),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

    learning_rate = 2e-5
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

    model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
    callbacks = myCallback()
    model.fit(train_set, epochs=1000, callbacks=callbacks)

    return model
    

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('mae') < 0.11:
            print("\nReached 0.11 MAE so cancelling training!")
            self.model.stop_training = True

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

def solution_B5():
    # Ganti dengan path lokal file CSV Anda
    file_path = 'goldstock_closing.csv'
    
    time_step = []
    temps = []

    with open(file_path) as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        next(reader)  # Skip header
        step = 0
        for row in reader:
            temps.append(float(row[2]))
            time_step.append(step)
            step += 1

    series = np.array(temps)
    
    # Normalisasi data
    min_val = np.min(series)
    max_val = np.max(series)
    series -= min_val
    series /= max_val
    time = np.array(time_step)

    # Split data
    split_time = 2000
    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]

    # Dataset parameters
    window_size = 64
    batch_size = 256
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation="relu", padding='causal', input_shape=[None, 1]),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

    learning_rate = 2e-5
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

    model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
    callbacks = myCallback()
    model.fit(train_set, epochs=1000, callbacks=[callbacks])

    return model, series, min_val, max_val

In [5]:
def predict_future(model, series, min_val, max_val, time, window_size, prediction_steps):
    forecast = []
    input_eval = series[-window_size:]
    for _ in range(prediction_steps):
        input_eval = input_eval.reshape((1, -1, 1))  # Reshape untuk input model
        prediction = model.predict(input_eval)[0, 0]  # Ambil prediksi dari array 2D
        forecast.append(prediction)
        input_eval = np.append(input_eval[1:], prediction)
    # Invers normalisasi untuk hasil prediksi
    forecast = np.array(forecast) * (max_val - min_val) + min_val
    return forecast


    # Jalankan fungsi dan simpan hasilnya
    model, series, min_val, max_val, time = solution_B5()
    
    # Prediksi 100 hari ke depan
    future_steps = 100
    forecast = predict_future(model, series, min_val, max_val, time, window_size=64, prediction_steps=future_steps)
    
    # Plot hasil prediksi
    plt.figure(figsize=(10, 6))
    plt.plot(time, series * (max_val - min_val) + min_val, label='Historical Data')  # Convert series back to original scale
    plt.plot(np.arange(len(series), len(series) + future_steps), forecast, label='Forecast', color='red')
    plt.xlabel('Time')
    plt.ylabel('Temperature')
    plt.legend()
    plt.show()
    

In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('mae') < 0.11:
            print("\nReached 0.11 MAE so cancelling training!")
            self.model.stop_training = True

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

def solution_B5():
    # Ganti dengan path lokal file CSV Anda
    file_path = 'goldstock_closing.csv'
    
    time_step = []
    temps = []

    with open(file_path) as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        next(reader)  # Skip header
        step = 0
        for row in reader:
            temps.append(float(row[2]))
            time_step.append(step)
            step += 1

    series = np.array(temps)
    
    # Normalisasi data
    min_val = np.min(series)
    max_val = np.max(series)
    series -= min_val
    series /= max_val
    time = np.array(time_step)

    # Split data
    split_time = 2000
    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]

    # Dataset parameters
    window_size = 64
    batch_size = 256
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation="relu", padding='causal', input_shape=[None, 1]),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

    learning_rate = 2e-5
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

    model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
    callbacks = myCallback()
    model.fit(train_set, epochs=1000, callbacks=[callbacks])

    return model, series, min_val, max_val

# Jalankan fungsi dan simpan hasilnya
model, series, min_val, max_val = solution_B5()

def predict_future(model, series, window_size, prediction_steps):
    forecast = []
    input_eval = series[-window_size:]
    for _ in range(prediction_steps):
        input_eval = input_eval.reshape((1, -1, 1))  # Reshape untuk input model
        prediction = model.predict(input_eval)[0, 0]  # Ambil prediksi dari array 2D
        forecast.append(prediction)
        input_eval = np.append(input_eval[0, 1:, 0], prediction)  # Update input untuk langkah berikutnya
    return forecast

# Prediksi 100 hari ke depan
window_size = 64
future_steps = 100
forecast = predict_future(model, series, window_size, future_steps)

# Invers normalisasi untuk hasil prediksi
forecast = np.array(forecast) * (max_val - min_val) + min_val

# Plot hasil prediksi
plt.figure(figsize=(10, 6))
plt.plot(time, temps, label='Historical Data')
plt.plot(np.arange(len(temps), len(temps) + future_steps), forecast, label='Forecast', color='red')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.legend()
plt.show()

IndexError: list index out of range

In [14]:
model, series, min_val, max_val = solution_B5()

# Prediksi 100 hari ke depan
def predict_future(model, series, window_size, prediction_steps):
        forecast = []
        input_eval = series[-window_size:]
        for _ in range(prediction_steps):
            input_eval = input_eval.reshape((1, -1, 1))  # Reshape untuk input model
            prediction = model.predict(input_eval)[0, 0]  # Ambil prediksi dari array 2D
            forecast.append(prediction)
            input_eval = np.append(input_eval[1:], prediction)
        return forecast

# Prediksi 100 hari ke depan
future_steps = 100
forecast = predict_future(model, series, window_size, future_steps)
    
# Invers normalisasi untuk hasil prediksi
forecast = np.array(forecast) * (max_val - min_val) + min_val
    
# Plot hasil prediksi
plt.figure(figsize=(10, 6))
plt.plot(time, temps, label='Historical Data')
plt.plot(np.arange(len(temps), len(temps) + future_steps), forecast, label='Forecast', color='red')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.legend()
plt.show()

Epoch 1/1000
8/8 [==============================] - 5s 171ms/step - loss: 0.0178 - mae: 0.1541
Epoch 2/1000
8/8 [==============================] - 1s 152ms/step - loss: 0.0178 - mae: 0.1538
Epoch 3/1000
8/8 [==============================] - 1s 169ms/step - loss: 0.0177 - mae: 0.1532
Epoch 4/1000
8/8 [==============================] - 1s 155ms/step - loss: 0.0176 - mae: 0.1526
Epoch 5/1000
8/8 [==============================] - 1s 162ms/step - loss: 0.0175 - mae: 0.1519
Epoch 6/1000
8/8 [==============================] - 1s 155ms/step - loss: 0.0174 - mae: 0.1513
Epoch 7/1000
8/8 [==============================] - 1s 155ms/step - loss: 0.0173 - mae: 0.1507
Epoch 8/1000
8/8 [==============================] - 2s 176ms/step - loss: 0.0172 - mae: 0.1501
Epoch 9/1000
8/8 [==============================] - 2s 208ms/step - loss: 0.0171 - mae: 0.1495
Epoch 10/1000
8/8 [==============================] - 2s 217ms/step - loss: 0.0170 - mae: 0.1490
Epoch 11/1000
8/8 [==============================

TypeError: cannot unpack non-iterable Sequential object

In [ ]:
pip install tensorflow

In [ ]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
import urllib

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('mae')<0.15):
      print("\nReached 15% mae so cancelling training!")
      self.model.stop_training = True

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)